In [1]:
import preprocessing as pp
import pandas as pd
import numpy as np
import time 
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

In [2]:
# Read and Preprocess the data
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# Customer Clustering
customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=60, size_min=4, size_max=400, random_state=42)

agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)


n_clus_scene = 3

df,scene_cluster_center_df = pp.constrained_kmeans_clustering(agg_dem_cus_period_scene_df, n_clusters=n_clus_scene, 
                                                                size_min=np.floor(len(agg_dem_cus_period_scene_df.columns)/n_clus_scene), 
                                                                size_max=len(agg_dem_cus_period_scene_df.columns), random_state=42)
agg_dem_cus_period_clus_scene_df = pp.agg_scene_df(agg_dem_cus_period_scene_df, scene_cluster_center_df)

/Users/thee/miniconda/envs/trm_conda/lib/python3.11/site-packages/k_means_constrained/k_means_constrained_.py:464: RuntimeWarning: invalid value encountered in cast
  costs = np.around(costs * 1000, 0).astype('int32')  # Times by 1000 to give extra precision


In [ ]:
agg_dem_cus_period_clus_scene_df

0             1             2
Cluster ProductIndex PeriodIndex                                          
0       0            0             3392.333333   3407.941176   3436.545455
                     1             6137.636364   6075.647059   6419.848485
                     2             9749.242424   9455.617647  10224.909091
                     3            12481.575758  12187.794118  13492.484848
                     4            16986.969697  16596.558824  18524.606061
...                                        ...           ...           ...
59      3            5             5203.606061   5214.117647   5898.272727
                     6             6052.272727   5793.735294   6655.727273
                     7             6815.545455   6637.941176   7926.212121
                     8             7685.636364   7395.117647   9193.909091
                     9             8657.575758   8300.882353   9935.212121

[2400 rows x 3 columns]

: 

In [ ]:
time_limit_s = 3600

model = xp.problem(name= 'MEWLP_Stochastics')
# Sets and Notation
S = list(supplier_df.index) # Supplier Index
S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

W = list(candidate_df.index) # Warehouse Index
C = list(cluster_center_df.index) # Cluster Index
P = list(agg_dem_cus_period_clus_scene_df.reset_index()['ProductIndex'].unique())
T = list(agg_dem_cus_period_clus_scene_df.reset_index()['PeriodIndex'].unique())
Phi = list(agg_dem_cus_period_clus_scene_df.columns)

# Output Variables
x = np.array([xp.var(f'x_{w}_{c}_{t}_{xi}', vartype = xp.binary) for w in W for c in C for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(T), len(Phi))
y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T), len(Phi))
z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))
a = np.array([xp.var(f'a_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(C), len(P), len(T), len(Phi))
b = np.array([xp.var(f'b_{w}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(P), len(T), len(Phi))

model.addVariable(x, y, o, v, z, a, b)

# Constraints 
for w in W:
    for t in T:
        model.addConstraint(xp.constraint(o[w] >= y[w,t]))
        if t != 0:
            model.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

        for c in C:
            for xi in Phi:
                model.addConstraint(xp.constraint(x[w,c,t,xi] <= y[w,t]))  

for xi in Phi:
    for t in T:
        for w in W:
            Capacity_W = candidate_df.loc[w,'Capacity']
            model.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t,xi] for c in C for p in P) <= Capacity_W * y[w,t]))
        for c in C:
            model.addConstraint(xp.constraint(xp.Sum(x[w,c,t,xi] for w in W) == 1))

for xi in Phi:
    for t in T:
        for c in C:
            for p in P:
                Demand = agg_dem_cus_period_scene_df.loc[(c,p,t),xi]
                for w in W:
                    model.addConstraint(xp.constraint(v[w,c,p,t,xi] + a[c,p,t,xi] == Demand * x[w,c,t,xi]))

for xi in Phi:
    for t in T:
        for w in W:
            for p in P:
                model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) + b[w,p,t,xi] == xp.Sum(v[w,c,p,t,xi] for c in C)))
                
for s in S:
    Capacity_S = supplier_df.loc[s,'SupplierCapacity']
    for t in T:
        model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))

Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)

equal_prob = 1/len(agg_dem_cus_period_clus_scene_df.columns)
ld = 10000000000
gm = 10000000000/2
Recourse = 0
for xi in Phi:
    Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t,xi] for w in W for c in C for p in P for t in T)
    Pen_w_c = ld* xp.Sum(a[c,p,t] for c in C for p in P for t in T)
    Pen_w_s = gm* xp.Sum(b[w,p,t] for w in W for p in P for t in T)
    Recourse += equal_prob * (Tra_w_c_cost + Pen_w_c + Pen_w_s)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Recourse
model.addObjective(Setup_cost, Operating_cost, Tra_w_s_cost, Recourse)
model.setObjective(sense = xp.minimize)

model.setControl('miprelstop', 1e-3)
model.setControl('maxtime', time_limit_s)
tic_time = time.time()
# Solve the problem
model.solve()
toc_time = time.time()
solve_time = toc_time - tic_time
obj_value = model.getObjVal()

mip_gap_percent = 100*(obj_value - model.getAttrib('bestbound'))/obj_value
print(f'Solving Time: {solve_time}')
print(f'Objective Value: {obj_value}')
print(f'%Gaps: {mip_gap_percent}')


Multi-objective optimization with 1 solves
Starting multi-objective solve 1
[obj 1/1] FICO Xpress v9.3.5, Hyper, solve started 5:26:34, Feb 13, 2025
[obj 1/1] Heap usage: 2608MB (peak 2608MB, 636MB system)
[obj 1/1] Minimizing MILP MEWLP_Stochastics using up to 11 threads and up to 18GB memory, with these control settings:
[obj 1/1] MAXTIME = 3600
[obj 1/1] OUTPUTLOG = 1
[obj 1/1] MIPRELSTOP = .001
[obj 1/1] Original problem has:
[obj 1/1]    4036690 rows      4258040 cols     19231520 elements    796840 entities
[obj 1/1] Presolved problem has:
[obj 1/1]     858880 rows      1075504 cols      7256990 elements    789504 entities
[obj 1/1] LP relaxation tightened
[obj 1/1] Presolve finished in 105 seconds
[obj 1/1] Heap usage: 3462MB (peak 5897MB, 636MB system)
[obj 1/1] 
[obj 1/1] Coefficient range                    original                 solved        
[obj 1/1]   Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 1.74e-02,  3.24e+03]
[obj 1/1]   RHS and bounds [min,max] : [ 1.0